In [ ]:
import pandas as pd
import numpy as np
import torch
# 进度条库
from tqdm import trange

# load data from csvfile 
# train_data = pd.read_csv(r".\3DTensor-basedDL4OptionPricing\data_ipynb\train_data.csv")
test_data = pd.read_csv(r".\3DTensor-basedDL4OptionPricing\data_ipynb\lewis_fft_results.csv")

train_input = []
train_label = []
seq_len = 10

In [ ]:
num = train_data['optID'].unique().shape[0]

In [3]:
print(num)

600


所以说，只有**train_data.csv**中也只有600支期权

In [ ]:
# .shape[0] 获取唯一optID的数量（去重后的）
for i in trange(train_data['optID'].unique().shape[0]):
    for index in range(train_data.loc[(train_data['optID'] == train_data['optID'].unique()[i])].shape[0] - seq_len+1):
        # 创建一个 3x10x5的零张量 表示每个输入数据的形状 其中3是通道数channels, 10是观察数， 5是变量(Delta Gamma Theta Vega Rho)
        # （3,10，5）--->(C,N,D)
        # 其中C=3 channels;D=5 number of variables;N=10 number of observations
        # unique()[i]取出第i个optID
        # train_data.loc[(train_data['optID'] == train_data['optID'].unique()[i])] 筛选出train_data里optID等于
        # 当前unique()[i]的所有数据
        # .shape[0]获取行数
        # 3 channels -- fundamental data: spot price | strike price | day to expire | call or put | iV
        # data of price: previous settle price | settle price change | theory price | theory margin | inventory(添头，为保持数据维度)
        # data of greeks：Delta Gamma Theta Vega Rho
        input = torch.zeros((3, 10, 5))
        tmp_input = np.array(train_data.loc[(train_data['optID'] == train_data['optID'].unique()[i])][index:index+seq_len])
        # tmp_input[0:seq_len, 3:8] 提取的是 第4列到第8列的数据， 对应着
        # inventory(k)库存 contractType dayToExpire iv strikeprice
        # 创建一个 3x10x5的零张量 表示每个输入数据的形状 其中3是通道数channels, 10是观察数， 5是变量(Delta Gamma Theta Vega Rho)
        # （3,10，5）--->(C,N,D)
        # 其中C=3 channels;D=5 number of variables;N=10 number of observations（用过去十天的期权数据预测期权数据）
        input[0, :] = torch.tensor(np.array(tmp_input[0:seq_len, 3:8], dtype=np.float64))
        # 4:8--inventory(k)//contractType//dayToExpire(days)//iv//strikeprice
        # 拿optID=10001025举例，就是
        ''' day1: [3.275, 0, 72, 25.4, 2.651]
            day2: [3.495, 0, 71, 24.9, 2.651]
            ...
            day10: [3.76, 0, 54, 25.8, 2.651]
        '''
        input[1, :] = torch.tensor(np.array(tmp_input[0:seq_len, 8:13], dtype=np.float64))
        # 9:13---delta//gamma//rho//theta//vega 所以channel 2====greeks
        '''
            day1: [1, 0, 0.0052, -3.00E-04, 0]
            day2: [0.0442, 35.7966, 2.00E-04, 0, 0.0011]
            ...
            day10: [0.5114, 2.1162, 0.0019, -8.00E-04, 0.004]
        '''
        input[2, :] = torch.tensor(np.array(tmp_input[0:seq_len, 13:18], dtype=np.float64))
                #14:18----pre_settlePrice(前结算价)//settlePirce_chg(结算价变动)//spotPrice(现货价格)//theoryMargin(理论保证金)//theoryPrice(理论价格)
        '''
        整体张量input(3,10,5)为:
        input = [
    # 通道 0：基本期权数据
    [
        [3.275, 0, 72, 25.4, 2.651],  # 第 1 天
        [3.495, 0, 71, 24.9, 2.651],  # 第 2 天
        ...
        [3.76, 0, 54, 25.8, 2.651]    # 第 10 天
    ],
    # 通道 1：希腊字母数据
    [
        [1, 0, 0.0052, -3.00E-04, 0],  # 第 1 天
        [0.0442, 35.7966, 2.00E-04, 0, 0.0011],  # 第 2 天
        ...
        [0.5114, 2.1162, 0.0019, -8.00E-04, 0.004]  # 第 10 天
    ],
    # 通道 2：其他金融数据
    [
        [0.1653, -20.4, 2.655, -0.1068, 0.0248],  # 第 1 天
        [0.1303, -13.4, 2.626, -0.1128, 0],  # 第 2 天
        ...
        [0.114, -8.9, 2.634, -0.0293, 0.0745]  # 第 10 天
    ]
]
        '''
        label = torch.tensor(np.array(tmp_input[seq_len-1, 18], dtype=np.float64))
        train_input.append(input)
        train_label.append(label)

因为一个31个滑动窗口，最后一个滑动窗口是 31th条数据-40th数据
最终的 train_input应该长这样

    train_input = [
    input_1,  # 第 1 天到第 10 天的数据
    input_2,  # 第 2 天到第 11 天的数据
    input_3,  # 第 3 天到第 12 天的数据
    ...
    input_31  # 第 31 天到第 40 天的数据
]

In [4]:
zeros = torch.zeros((len(train_input), 3, 10, 5))

In [5]:
print(zeros.shape[0])

0


In [6]:
print(len(train_input))

0


In [7]:
# 创建一个四维张量
zeros = torch.zeros((len(train_input), 3, 10, 5))
# zeros = torch.zeros(600x31, 3, 10, 5)(假设每个optID的数据都是40条) 但其实是33210个
for i in range(zeros.shape[0]):
    zeros[i] = train_input[i]
train_input = zeros
# 将train_input由列表转换为张量
# 现在的train_input应该长
# tensor(input_1, input_2, ..., input_33210) 其中input_1=[3,10,5]

In [9]:
print("Shape of the Training Input:", train_input.shape)

Shape of the Training Input: torch.Size([33210, 3, 10, 5])


In [10]:
train_label = torch.tensor(train_label)
print("Shape of the Training Label:", train_label.shape)

Shape of the Training Label: torch.Size([33210])


对test_data_csv做同样的操作，pandas+numpy-->list---pytorch---tensor

In [8]:
test_input = []
test_label = []

滑动窗口

In [ ]:
for i in trange(test_data['optID'].unique().shape[0]):
    for index in range(test_data.loc[(test_data['optID'] == test_data['optID'].unique()[i])].shape[0] - seq_len+1):
        input = torch.zeros((3, 10, 5))
        tmp_input = np.array(test_data.loc[(test_data['optID'] == test_data['optID'].unique()[i])][index:index+seq_len])
        input[0, :] = torch.tensor(np.array(tmp_input[0:seq_len, 3:8], dtype=np.float64))
        input[1, :] = torch.tensor(np.array(tmp_input[0:seq_len, 8:13], dtype=np.float64))
        input[2, :] = torch.tensor(np.array(tmp_input[0:seq_len, 13:18], dtype=np.float64))
        label = torch.tensor(np.array(tmp_input[seq_len-1, 18], dtype=np.float64))
        test_input.append(input)
        test_label.append(label)


数据转换为张量

In [25]:
zeros = torch.zeros((len(test_input), 3, 10, 5))

In [26]:
for i in range(zeros.shape[0]):
    zeros[i] = test_input[i]
test_input = zeros

In [18]:
print("Shape of the Test Input:", test_input.shape)

Shape of the Test Input: torch.Size([11386, 3, 10, 5])


In [27]:
test_label = torch.tensor(test_label)
print("Shape of the Test Label:", test_label.shape)

Shape of the Test Label: torch.Size([11386])


存储为pt

In [ ]:
#torch.save(train_input, r".\3DTensor-basedDL4OptionPricing\data_ipynb\torch-data-ipynb\train_input.pt")
#torch.save(train_label, r".\3DTensor-basedDL4OptionPricing\data_ipynb\torch-data-ipynb\train_label.pt")
torch.save(test_input, r".\3DTensor-basedDL4OptionPricing\data_ipynb\torch-data-ipynb\test_input(alpha=1.25).pt")
torch.save(test_label, r".\3DTensor-basedDL4OptionPricing\data_ipynb\torch-data-ipynb\test_label(alpha=1.25).pt")

通道0：基础期权数据 \
特征：[inventory(k), contractType, dayToExpire, iv, strikePrice] \
​通道1：希腊字母数据 \
特征：[delta, gamma, rho, theta, vega] \
​通道2：价格相关数据 \
特征：[pre_settlePrice, settlePrice_chg, spotPrice, theoryMargin, theoryPrice] 
### train_input.pt 和 test_input.pt
# 一个样本示例 (3,10,5)
[
    # 通道0 - 基础数据
    [
        [3.275, 0, 72, 25.4, 2.651],  # 第1天
        [3.495, 0, 71, 24.9, 2.651],  # 第2天
        ...                           # 第3-9天
        [3.76, 0, 54, 25.8, 2.651]    # 第10天
    ],
    
    # 通道1 - 希腊字母
    [
        [1, 0, 0.0052, -3.00E-04, 0],        # 第1天
        [0.0442, 35.7966, 2.00E-04, 0, 0.0011], # 第2天
        ...                                   # 第3-9天
        [0.5114, 2.1162, 0.0019, -8.00E-04, 0.004] # 第10天
    ],
    
    # 通道2 - 价格数据
    [
        [0.1653, -20.4, 2.655, -0.1068, 0.0248],  # 第1天
        [0.1303, -13.4, 2.626, -0.1128, 0],       # 第2天
        ...                                       # 第3-9天
        [0.114, -8.9, 2.634, -0.0293, 0.0745]     # 第10天
    ]
]

### train_label.pt 和 test_label.pt
# 对应于上面示例样本的标签
tensor(0.1316)  # 第10天的结算价